# Using the autoload & 1D barcode reader

| Summary | Image |
|--------|--------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"><li><b>Feature:</b> Autoload with integrated 1D barcode reader</li><li><b>Operation:</b> Automatically loads carriers and plates from the front-loading tray onto the STAR deck.</li><li><b>Barcode Recognition:</b> Scans 1D barcodes on carriers, plates, tube racks, and tip racks to confirm ID and placement.</li><li><b>Purpose:</b> Reduces manual deck verification by ensuring that physically loaded carriers match the expected layout.</li><li><b>Workflow Benefit:</b> Verification + Speeds up run setup through hands-free loading and automatic identification.</li><li><b>Best For:</b> High-throughput or frequently changing deck configurations.</li></ul> | <div style="width:320px; text-align:center;">![real_autoload](img/hamilton_star_autoload.png)<br><i>Figure: Hamilton STAR Autoload system</i></div> |


## Background: Architecture of Hamilton STAR(let) Autoload

![hamilton_star_overview](img/hamilton_autoload_overview.png)

## Setup

In [ ]:
from pylabrobot.liquid_handling import LiquidHandler, STARBackend
from pylabrobot.resources import STARDeck
from pylabrobot.resources import (
  TIP_CAR_480_A00,
  PLT_CAR_L5AC_A00,
  TIP_50ul,
  Cor_96_wellplate_360ul_Fb
)

star = STARBackend()
lh = LiquidHandler(backend=star, deck=STARDeck())
await lh.setup()

# assign a tip rack carrier
tip_carrier = TIP_CAR_480_A00(name="tip_carrier")
tip_carrier[1] = tip_rack = TIP_50ul(name="tip_rack")
lh.deck.assign_child_resource(tip_carrier, rails=10)

# assign a plate carrier
plt_carrier = PLT_CAR_L5AC_A00(name="plt_carrier")
plt_carrier[0] = plate = Cor_96_wellplate_360ul_Fb(name="plt")
lh.deck.assign_child_resource(plt_carrier, rails=30)

```{note}
Two starting points are possible:
1. Carriers have been moved directly on the deck.
2. Carriers have been left on the loading tray.

Here we assume we're starting with (1) Carriers have been moved directly on the deck.
```

## Querying autoload state

In [ ]:
await star.request_autoload_track()

## Sensing carriers

The autoload sled has a front-facing proximity sensor.
This sensor can be used to scan the entire loading tray to identify whether there are any carriers currently on the loading tray:

In [ ]:
await star.request_presence_of_carriers_on_loading_tray()

```{note}
The autoload detects the righ-most track occupied by a carrier!
```

Similarly, if you only want to investigate a single position this can be done too. 

In [ ]:
await star.request_presence_of_single_carrier_on_loading_tray(track=10)

The counterpart to checking for carriers on the loading tray is checking for presence of carriers on the liquid handler's deck:

In [ ]:
await star.request_presence_of_carriers_on_deck()

```{note}
`.request_presence_of_carriers_on_deck()` is technically not an 'autoload' command.
It uses the presence sensors at the back of the STAR(let) deck to identify whether a carrier is present.
i.e. there is no autoload movement involved, and it therefore works for STAR(let)s without an integrated barcode sensor too.
```

Together these `STAR` methods enable capturing a full picture of the state of carriers on your liquid handler.

## Moving autoload

In [ ]:
await star.move_autoload_to_track(10)

In [ ]:
await star.park_autoload()

## 1D Barcode reading
There are 2 main types of 1D barcodes one might encounter:
1. Carrier barcodes (orientation=vertical; located at the right-back of the carrier)
2. Container barcodes:
    - TipRack barcodes (orientation=horizontal)
    - Plate barcodes (orientation=horizontal)
    - Tube barcodes (orientation=vertical)


The fastest way to read your barcode *when* your carriers are already on the deck is to move the carrier out to the identification position:

In [ ]:
await star.take_carrier_out_to_identification_position(tip_carrier)
reading = await star.load_carrier_from_identification_position(
    barcode_reading=True,
    park_autoload_after=False
    )

Before reading barcodes it is now important to set the barcode symbology that you are expecting to read!

The following barcode symbologies can be detected by the system:
- ISBT standard
- Code 128 (subset B and C)
- Code 39
- Codabar
- Code 2 of 5 Interleaved
- UPC A/E
- JAN/EAN 8

For the highest reading safety Hamilton recommends to use barcode type `Code128 (subset B and C)`.

This is the default symbology chosen in PyLabRobot commands.
However, you can directly set or change the expected barcode symbology:

In [ ]:
await star.set_1d_barcode_type("Code 128 (Subset B and C)")

Next, readthe 

```{warning}
The 1D barcode scanner uses a laser at a fixed height.
This is especially important when reading horizontal barcodes.
The z-height of the laser (during horizonatal) barcode reading is `z=218` or 118 mm above the deck surface.
If your 1D barcode is not precisely positioned at this height, the 1D barcode reader cannot read your barcode.
To facilitate this height, use "DWP" carriers/MFX plate_holders for plates >40 mm in `size_z`, and "MP" carriers/MFX plate_holders for plates ~15 mm in `size_z`.
```

## Loading / Unloading carriers